In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import imutils
import glob
import argparse
import pandas as pd
import xlrd 
%matplotlib inline

In [2]:
def auto_contrast(img):
    height, width = img.shape[:2] 
    a_high=0
    a_low=255
    for i in img:
        for j in i:
            if j >= a_high:
                a_high = j
            if j<=a_low:
                a_low =j
    print(a_high)
    print(a_low)
    for i in range(height):
        for j in range(width):
            img[i][j] =  (img[i][j] -a_low)*255/(a_high-a_low)
    a_high=0
    a_low=255
    for i in img:
        for j in i:
            if j >= a_high:
                a_high = j
            if j<=a_low:
                a_low =j
    print(a_high)
    print(a_low)
    return img

In [3]:
#delute blood vessel in green chanel
def vessel_erase(scr):    
    img = scr.copy()
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    #cl1 = clahe.apply(img)
    kernel15 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))
    img_dilation = cv2.dilate(img, kernel15, iterations=1) 
    img_dilation_erosion = cv2.erode(img_dilation, kernel15, iterations=1)
    return img_dilation_erosion

In [1]:
def drawEllipse(img,contours,color,x,y):
    '''
    draw ellipse im image from contour
    Parameters:
        -img: draw ellipse in this image
        -contours: a contour you want to fit ellipse in
        -x,y: x and y of top left ROI in original image
    Returns:
        -displayimg: image output
    '''
    displayimg = img.copy()
    # Find the rotated rectangles and ellipses for each contour
    # https://docs.opencv.org/3.4/de/d62/tutorial_bounding_rotated_ellipses.html
    for i in contours:
        for j in i:
            j[0][0] = j[0][0]+x
            j[0][1] = j[0][1]+y
    minEllipse = [None]*len(contours)
    for i, c in enumerate(contours):
        if c.shape[0] > 5:
            minEllipse[i] = cv2.fitEllipse(c)

    for i, c in enumerate(contours):
        if c.shape[0] > 5:
            cv2.ellipse(displayimg, minEllipse[i], color, 2)

    return displayimg

In [5]:
def drawRect(img,contours):
    displayimg = img.copy()
    # Find the rotated rectangles and ellipses for each contour
    # https://docs.opencv.org/3.4/de/d62/tutorial_bounding_rotated_ellipses.html
    minRect = [None]*len(contours)
    for i, c in enumerate(contours):
        minRect[i] = cv2.minAreaRect(c)

    for i, c in enumerate(contours):
        color = (255,255,255)
        # rotated rectangle
        box = cv2.boxPoints(minRect[i])
        box = np.intp(box) #np.intp: Integer used for indexing (same as C ssize_t; normally either int32 or int64)
        cv2.drawContours(displayimg, [box], 0, color)

    return displayimg

In [6]:
def biggestContour(contours):
    contours_sizes = [(cv2.contourArea(cnt), cnt) for cnt in contours]
    biggest_contour = max(contours_sizes, key=lambda x: x[0])[1]
    return biggest_contour

In [42]:
def getImName(image_path,folder_path):
    """
    get image name by clean up image path
    Parameters:
        -image_path: path of image in the folder
        -folder_path: path of folder that contain images
    Returns:
        -name: name of the image
    """
    name = image_path.replace(folder_path.replace("/*.jpg","")+"\\","")
    print(name)
    return name

def getCenterROI(image_name,df):
    """
    get center ROI from image name and dataframe
    Parameters:
        -image_name: name of image that match with name in dataframe
        -df: dataframe that contain image name and ROI center coordinate
    Returns:
        -x,y: coordinate of ROI center
    """
    x = df.loc[image_name][0]
    y = df.loc[image_name][1]
    return x,y

def getROI(img,x,y,ROI_size):
    """
    crop square ROI from image with center x, y coordinate
    Parameters:
        -img: input image
        -x,y: center coordinate of ROI
        -ROI_size: size of ROI in pixel
    Returns:
        -imCrop: image of ROI
        -x1,y1: coordinate of top left corner of the ROI
        -x2,y2: coordinate of botton right corner of the ROI
    """
    x1 = int(x-(ROI_size/2))
    y1 = int(y-(ROI_size/2))
    x2 = int(x+(ROI_size/2))
    y2 = int(y+(ROI_size/2))
    imCrop = img[y1:y2,x1:x2]
    return imCrop,x1,y1,x2,y2

def readImgFolder(path):
    """
    read image form folder and create a list of images path in side the folder
    Parameters:
        -path: folder path (ex.'RAW_FUNDUS_INPUT/*.jpg')
    Returns:
        -image_list: list of images path in the folder
    """
    image_list = []
    for filename in glob.glob(path):
        image_list.append(filename)
    return image_list

def ShowResizedIm(img,windowname,scale):
    """
    opencv imshow resized image on a new window
    Parameters:
        -img: image
        -window: window name
        -scale: size of the display image will be divided by this value(ex. scale=2 will make image 2 time smaller)
    """
    cv2.namedWindow(windowname, cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    height, width = img.shape[:2]   #get image dimension
    cv2.resizeWindow(windowname,int(width/scale) ,int(height/scale))                    # Resize image
    cv2.imshow(windowname, img)                            # Show image

#=========USER START================
#folder path
img_folder_path = 'RAW_FUNDUS_INPUT/*.jpg'
ROI_path = 'ROI.xlsx'
ROI_size = 400
save_path = 'K mean/'
#=========USER END================

#create dataframe of ROI.xlsx
df = pd.read_excel(ROI_path, index_col=0)
#crate list of image path in the folder
img_path_list = readImgFolder(img_folder_path)
for img_path in img_path_list:
    #read only name
    img_name = getImName(img_path,img_folder_path)
    #read x,y coordinate from ROI.xlsx
    x_ROI,y_ROI = getCenterROI(img_name,df)
    #read image
    img = cv2.imread(img_path)
    b,g,r = cv2.split(img)
    #get ROI image, x1, y1, x2, and y2
    img_ROI,x1_ROI,y1_ROI,x2_ROI,y2_ROI = getROI(g,x_ROI,y_ROI,ROI_size)
    #=====================PUT YOUR CODE HERE===========================
    opening,disc_contours2,cup_contours2 = applyKmean(img_ROI,3)
    img_out = drawEllipse(drawEllipse(img,disc_contours2,(255,0,0),x1_ROI,y1_ROI),
                                 cup_contours2,(0,255,0),x1_ROI,y1_ROI)
    cv2.imwrite(save_path+img_name,img_out)
    #==================================================================

267_001.jpg
267_002.jpg
267_008.jpg
267_009.jpg
267_010.jpg
267_011.jpg
267_016.jpg
267_017.jpg
267_018.jpg
267_019.jpg
267_021.jpg
267_024.jpg
267_025.jpg
267_026.jpg
267_027.jpg
267_028.jpg
267_031.jpg
267_032.jpg
267_033.jpg
267_036.jpg
267_037.jpg
267_038.jpg
267_041.jpg
267_042.jpg
267_045.jpg
267_046.jpg
267_049.jpg
267_050.jpg
267_051.jpg
267_053.jpg
267_056.jpg
267_064.jpg
267_067.jpg
267_068.jpg
267_069.jpg
267_070.jpg
267_073.jpg
267_074.jpg
267_075.jpg
267_076.jpg
267_077.jpg
267_078.jpg
267_079.jpg
267_081.jpg
267_082.jpg
267_083.jpg
267_084.jpg
267_085.jpg
267_086.jpg
267_087.jpg
267_089.jpg
267_091.jpg
267_092.jpg
269_001.jpg
269_002.jpg
269_003.jpg
269_004.jpg
269_005.jpg
269_006.jpg
269_007.jpg
269_008.jpg
269_009.jpg
269_010.jpg
269_011.jpg
269_012.jpg
269_013.jpg
269_014.jpg
269_015.jpg
269_016.jpg
269_017.jpg
269_018.jpg
269_019.jpg
269_020.jpg
269_021.jpg
269_022.jpg
269_023.jpg
269_024.jpg
269_025.jpg
269_026.jpg
269_027.jpg
269_028.jpg
269_029.jpg
269_030.jpg
269_

In [175]:
img = cv2.imread(img_source)
width = int(img.shape[1])
height = int(img.shape[0])
img = cv2.resize(img, (width,height), interpolation = cv2.INTER_AREA)

#color image
#cut out white text
img = img[int(100):int(-1), int(0):int(-1)]

#color image
b,g,r = cv2.split(img)

ShowResizedIm(np.hstack((b,g,r)),"img1",3)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [176]:
#g = vessel_erase(g)
ShowResizedIm(g,"img1",3)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [177]:
#find bright spot
# apply a Gaussian blur to the image then find the brightest
# region
imblur = cv2.GaussianBlur(g,(21,21),0)
(minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(imblur)
print(maxLoc)
image = g.copy()
cir = cv2.circle(image,maxLoc,50,(255, 255, 255), 3)

ShowResizedIm(np.hstack((g,cir)),"img1",3)


cv2.waitKey(0)
cv2.destroyAllWindows()

(1744, 926)


In [178]:
# Crop image
# https://www.learnopencv.com/how-to-select-a-bounding-box-roi-in-opencv-cpp-python/
crop_width =380
imCrop = g[int(maxLoc[1]-(crop_width/2)):int(maxLoc[1]+(crop_width/2)), int(maxLoc[0]-(crop_width/2)):int(maxLoc[0]+(crop_width/2))]
cv2.imshow("Image", imCrop)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
def applyKmean(imCrop,K):
    imCrop = cv2.GaussianBlur(imCrop,(21,21),0)
    Z = imCrop.reshape((-1,1))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 1000, 0.001)
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((imCrop.shape))
    #collect intesity value of background, disc, and cup
    inten_bg = center.min()
    inten_cup = center.max()
    for i in center: 
        if i > center.min() and i < center.max():
            inten_disc = i[0]
    #morphology
    kernel = np.ones((10,10),np.uint8)
    opening = cv2.morphologyEx(res2, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    #disable the cup cluster
    disc_cluster = opening.copy()
    disc_cluster[disc_cluster == inten_cup] = inten_disc
    ret, disc_thresh = cv2.threshold(disc_cluster, inten_bg , 255, 0)
    __,disc_contours, disc_hierarchy = cv2.findContours(disc_thresh, cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)

    disc_contours2 = [biggestContour(disc_contours)]
    cv2.drawContours(disc_cluster, disc_contours2, -1, (0,255,0), 3)
    #disable the disc cluster
    cup_cluster = opening.copy()
    cup_cluster[cup_cluster == inten_disc] = inten_bg
    ret, cup_thresh = cv2.threshold(cup_cluster, inten_bg , 255, 0)
    __,cup_contours, cup_hierarchy = cv2.findContours(cup_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cup_contours2 = [biggestContour(cup_contours)]
    cv2.drawContours(cup_cluster, cup_contours2, -1, (0,255,0), 3)
    return opening,disc_contours2,cup_contours2


In [180]:
# Apply K mean
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html
imCrop = cv2.GaussianBlur(imCrop,(21,21),0)
Z = imCrop.reshape((-1,1))
# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 1000, 0.001)
K = 3
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((imCrop.shape))

#collect intesity value of background, disc, and cup
inten_bg = center.min()
inten_cup = center.max()
for i in center: 
    if i > center.min() and i < center.max():
        inten_disc = i[0]
    


cv2.imshow('img',np.hstack((imCrop,res2)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [181]:
#morphology
# https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html
kernel = np.ones((10,10),np.uint8)
opening = cv2.morphologyEx(res2, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
cv2.imshow('img',np.hstack((res2,opening)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [182]:
#disable the cup cluster
disc_cluster = opening.copy()
disc_cluster[disc_cluster == inten_cup] = inten_disc
ret, disc_thresh = cv2.threshold(disc_cluster, inten_bg , 255, 0)
disc_contours, disc_hierarchy = cv2.findContours(disc_thresh, cv2.RETR_TREE,  cv2.CHAIN_APPROX_SIMPLE)

disc_contours2 = [biggestContour(disc_contours)]
cv2.drawContours(disc_cluster, disc_contours2, -1, (0,255,0), 3)

# show the image
cv2.imshow('img',disc_cluster)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [183]:
#disable the disc cluster
cup_cluster = opening.copy()
cup_cluster[cup_cluster == inten_disc] = inten_bg
ret, cup_thresh = cv2.threshold(cup_cluster, inten_bg , 255, 0)
cup_contours, cup_hierarchy = cv2.findContours(cup_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

cup_contours2 = [biggestContour(cup_contours)]
cv2.drawContours(cup_cluster, cup_contours2, -1, (0,255,0), 3)

# show the image
cv2.imshow('img',cup_cluster)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [184]:
cv2.imshow('img',np.hstack((opening,drawEllipse(drawEllipse(imCrop,disc_contours2),cup_contours2))))
cv2.waitKey(0)
cv2.destroyAllWindows()